In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="white", color_codes=True)
import matplotlib.pyplot as plt
import matplotlib as mpl
font = {"family":"Ricty"}
mpl.rc('font', **font)

In [ ]:
# とりあえずデータ読み込め！
# 選挙結果のデータ
pres08 = pd.read_csv('officialSource/qss-inst-master/PREDICTION/betting-markets/data/pres08.csv')
# 世論調査のデータ
polls08 = pd.read_csv('officialSource/qss-inst-master/PREDICTION/betting-markets/data/polls08.csv')
# それぞれのデータ内に、オバマとマケインの得票率(もしくは支持率)の差を「margin」カラムとして保存
pres08['margin'] = pres08['Obama'] - pres08['McCain']
polls08['margin'] = polls08['Obama'] - polls08['McCain']

# 同じ州の同じ中間日に世論調査が行われているのか？を確認
polls08.groupby(['state','middate']).count()
# 行われてるので、平均をとる必要がある

# pythonでのDate型ってこうやるらしい
polls08['middate'] = pd.to_datetime(polls08['middate'])
# middate は最大でも20081101
polls08['middate'].max()

# 試しに色々計算させてみる
pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-11')
pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-16')
import math
abs(pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-16'))
type(pd.to_datetime('2019-01-12') - pd.to_datetime('2019-01-11'))

# 選挙日までの日数をカラムとして保存
polls08['DaysToElection'] = pd.to_datetime('2008-11-04') - polls08['middate']

# DaysToElectionを小さい方から並べたランクナンバーを振る
polls08 = polls08.assign(
    rankMin=polls08.groupby(by='state')['DaysToElection'].rank(method='min')
)
# 最も選挙日に近い世論調査に限定して、オバマとマケインの差の平均をとり、それを予測値カラム(カラム名：pred)として保存
poll = polls08[polls08['rankMin']==1].groupby(by='state', as_index=False).mean()[['state','margin']]
poll = poll.rename(columns={'margin':'pred'})
# 州の名前をつけ直す
poll = pd.merge(poll, pres08, how='left', on='state')
# 誤差カラムを追加
poll['error'] = poll['margin'] - poll['pred']
# 誤差の平均を算出(＝平均予測誤差)
poll['error'].mean()
# root mean squre(2乗平均平方根)で誤差を見てみる
math.sqrt((poll['error']**2).mean())

In [ ]:
myfig, ax1 =plt.subplots(ncols=1, nrows=1, figsize=(8,6))
ax1 = sns.distplot(poll['error'], ax=ax1, bins=[-20, -15, -10, -5, 0, 5, 10, 15, 20])
ax1.axvline(poll['error'].mean())
ax1.text(x=poll['error'].mean(), y=0.06, s='平均誤差')
myfig.show()

世論調査結果と、実際の選挙結果の関係性をビジュアルで把握するため、x軸に世論調査結果、y軸に選挙結果をプロットする

In [ ]:
myfig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
# 一旦グラフを描かせるために、アルファゼロでプロットしちゃう
ax1 = sns.scatterplot(x='pred', y='margin', data=poll, alpha=0)
# プロットの位置に、州名を文字としておいていく
for index, row in poll.iterrows():
    print(row['pred'], row['margin'], row['state'])
    ax1.text(x=row['pred'], y=row['margin'], s=row['state'], ha='center', va='center')
# 45度線を描く
ax1 = sns.lineplot(x=[-40, 80], y=[-40, 80], ax=ax1, color='gray')
# seaborn経由で線グラフを破線にしたい場合、これ以外に方法ないんだろうか・・
ax1.lines[0].set_linestyle('--')
ax1.axvline(0, linestyle="-", color='gray')
ax1.axhline(0, color='gray')
myfig.show()

今回の選挙結果予測においては、各候補者の支持率差分を選挙結果(得票率)の予測値として使用したが、細かな値の差はさほど重要ではなく、大事なのは、「オバマ勝利と予測して、実際本当にオバマが勝利したかどうか？」といった、「勝った」「負けた」の判断を正しく予測できたか？になる。
そこで、今回の世論調査が、選挙結果を予測するものとして間違っていた州を把握してみる。

In [ ]:
# 予測値、結果、ともに、正負の符号が一致していれば、勝敗の予測が正しかったことになるので、
# 符号が異なっていた州名をリストアップしてみる
poll[(poll['pred'] > 0) != (poll['margin'] > 0)]

どの州も、支持率の差が1％ポイント程度しかない接戦だったことがわかる。

世論調査が、オバマへの選挙人表の数を予測できていたのかを確認してみる。

In [ ]:
# その州の選挙人数は、得票率が高い候補者に投票するので、以下のような計算で算出できる
poll[poll['margin'] > 0]['EV'].sum()

# 世論調査での選挙人数
poll[poll['pred'] > 0]['EV'].sum()

実際の結果は364人、世論調査では349人と、15人の差が出ているが、勝利の境目が270人であるので、「勝つ」「負ける」の予測としては、世論調査も正しく予測できていた。

実際の得票率を世論調査の支持率で予測できるかどうか(＝世論調査は正しいのかどうか？)と、世論調査の変遷を確かめる方法を考えてみる。
pollsUS08.csvには、選挙期間中の世論調査のデータが入っている。この時系列のデータを用いて、以下のような方法での検証を試みる。

- 当該日から7日前までの移動平均値を当該日の値として算出
- 世論調査が正しく集められているのであれば、選挙当日までの世論の変遷は時系列グラフで読み解くことができ、かつ、最終的に投票日の選挙結果と全日の世論調査の支持率の結果がニアリーになっているはず。

In [ ]:
# 何はともあれデータ読み込み
pollsUS08 = pd.read_csv('officialSource/qss-master/PREDICTION/pollsUS08.csv')
pollsUS08.dtypes
# 日付文字列をpandas.datetime64に変換
pollsUS08['middate'] = pd.to_datetime(pollsUS08['middate'])
pollsUS08.dtypes

# 選挙前90日間のデータを作っていく
# 一旦、何日分のデータがあるのかを確認
pollsUS08["middate"].describe()
# 選挙日は2008-11-04 なので、前日までのデータが入っていることを確認。

# 日付演算、「1日前」「3ヶ月後」などは、下記のように timedelta型のインスタンスで計算させる。
pollsUS08.middate[0] - pd.to_timedelta(1,'day')
from dateutil.relativedelta import relativedelta
# ○ヶ月後、みたいな処理をする場合、月末日を考慮してもらうためには、dateutil.relativedelta を使うと良い
pollsUS08.middate[0] + relativedelta(months=3)
pd.to_datetime('2019-02-28') - relativedelta(months=1)
# 下記のような計算の場合、ちゃんと考慮して2月の末日を返してくれる。SQLのADD_MONTHS関数的なやつ
pd.to_datetime('2019-01-31') + relativedelta(months=1)
pd.to_datetime('2019-01-31') + relativedelta(days=3)

# 結論、「○日後」「○ヶ月前」みたいな計算は、dateutil.relativedelta 使っとけ、って感じですかね。

# 本論に戻りまして。
# pollsUS08のうち、選挙日2008-11-04 から90日前までのデータに限定して、日毎に支持率の平均を計算していく。
temp = pollsUS08[pollsUS08['middate'] > (pd.to_datetime('2008-11-04') - relativedelta(days=90))].copy()

# あ。。。SQLのウィンドウ関数的なんがわからんので、検証。
temp.rolling('5D', on='middate').mean()
# んーー、ウィンドウ関数を判断する対象となるカラムは、重複データがあると動かないらしい。まあ、そうか・・。

# しょうがないので、一旦日別のデータで下準備する
temp2 = temp.groupby(by='middate', as_index=False).sum()
temp2 = temp2.rename(columns={'McCain': 'SUM_McCain', 'Obama':'SUM_Obama'})
temp2 = temp2.merge(temp.groupby(by='middate', as_index=False).count()[['middate','Pollster']], how='left', on='middate')
temp2 = temp2.rename(columns={'Pollster':'count'})
# からの、7日間の移動平均。本では当該日を含まない7日前のデータで平均を取ってるけど、
# ちょっとめんどくさいし、普通の移動平均チックな考え方だと当該日含んでるので、当該日を含む7日前の
# 平均値を取る
temp3 = temp2.rolling('7D', on='middate').sum()
temp3['MEAN_McCain'] = temp3['SUM_McCain']/ temp3['count']
temp3['MEAN_Obama'] = temp3['SUM_Obama']/ temp3['count']
# 選挙結果のレコードを挿入
s = pd.Series([pd.to_datetime('2008-11-04'), np.nan, np.nan, 1, 45.65, 52.93], index=temp3.columns)
temp3 = temp3.append(s, ignore_index=True)
myfig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax1 = sns.lineplot(data=temp3, x='middate', y='MEAN_McCain', label='McCain', ax=ax1)
ax1 = sns.lineplot(data=temp3, x='middate', y='MEAN_Obama', label='Obama', ax=ax1)
myfig.show()

仮説：1秒未満での顔の見た目で、民主党の候補者のほうが能力が高いと答えた割合で、民主党候補の選挙結果が予測できる

この仮説を確かめる前に、民主党候補者の見た目スコアと、実際の得票率の差分の分布の関係を可視化してみる

In [ ]:
# まず読み込み
face = pd.read_csv('officialSource/qss-master/PREDICTION/face.csv')
# 得票率の差分を出す
face['d_share'] = face['d.votes'] / (face['d.votes'] + face['r.votes'])
face['r_share'] = face['r.votes'] / (face['d.votes'] + face['r.votes'])
face['diff_share'] = face['d_share'] - face['r_share']
# 描画していく
myfig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax1 = sns.scatterplot(data=face, x='d.comp', y='diff_share', hue='w.party', ax=ax1)
ax1.set_xlim([0, 1])
ax1.set_ylim([-1, 1])
ax1.xaxis.label.set_text('民主党候補の見た目能力スコア')
ax1.yaxis.label.set_text('得票率における民主党候補のマージン')
ax1.title.set_text('見た目の能力と得票率')
myfig.show()

In [ ]:
# 相関係数を出す 手法その1
# pandas の相関係数は、メソッドが選べて、各列総当たり戦をしてくれる。
# Null除去もリストワイズ的ではなく、ちゃんと使う列でNullがあった時のみ、除去としてくれている。
face[["d.comp", "diff_share"]].corr()

# 相関係数を出す 手法その2 scipy で出すと帰無仮説(無相関)のp値も出してくれる
import scipy.stats
scipy.stats.pearsonr(face["d.comp"], face['diff_share'])
# 相関係数が0.4327743... 、p値がすっごい小さい。
# 弱い相関はありそうね。
